In [1]:
# This notebook is divided into two main parts:

# Part 1: Model Validation
# In this section, the focus is on validating pre-trained models. 
# It involves assessing the performance of models that have already been trained, 
# using predefined parameters such as time index ranges for the validation dataset. 
# The goal is to evaluate the predictive accuracy of these models on test data.

# Part 2: Model Training
# This section is dedicated to training your own models. 
# It guides you through the process of setting up and configuring a new model, 
# including defining its parameters and architecture. 
# You will train the model using your dataset, allowing for experimentation 
# with different configurations to develop a model that best suits your data and prediction objectives.


In [2]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import tensorflow as tf 
import tensorboard as tb 
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import copy
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss,MAE,MAPE,RMSE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
import pandas as pd

data = pd.read_excel('./dataset/EMNN Decomposed Data.xlsx')
data["day"] = data["day"].astype(str)
data["Brent"] = data["WTI"].astype("float64")
data["month"] = data["month"].astype(str)
data["weekday"] = data["weekday"].astype(str)

In [4]:
# create dataloaders for model
batch_size = 256  # set this between 32 to 128


# check if GPU is available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# move dataloaders to device


## Part 1: Model Validation

In [25]:
# IMF 1

In [19]:
# Used for storing the trained model
pretrained_model_paths=['.\\TFT_saved_models\\best_model_epoch=41-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v84.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v85.ckpt', '.\\TFT_saved_models\\best_model_epoch=19-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=26-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=23-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v86.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v87.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v88.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v89.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=32-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v90.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v20.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v91.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v31.ckpt', '.\\TFT_saved_models\\best_model_epoch=39-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v93.ckpt', '.\\TFT_saved_models\\best_model_epoch=1-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v32.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v94.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v95.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v96.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v97.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v98.ckpt', '.\\TFT_saved_models\\best_model_epoch=44-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v99.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v100.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v30.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v101.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v102.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v103.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v104.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=40-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v105.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v106.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v107.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=24-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v108.ckpt', '.\\TFT_saved_models\\best_model_epoch=44-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v109.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v110.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v111.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v112.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v113.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v114.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v115.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v116.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v31.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v32.ckpt', '.\\TFT_saved_models\\best_model_epoch=44-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v117.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v118.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v41.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v125.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v42.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v126.ckpt', '.\\TFT_saved_models\\best_model_epoch=35.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v43.ckpt', '.\\TFT_saved_models\\best_model_epoch=23-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v127.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v128.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v44.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v129.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v22.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v130.ckpt', '.\\TFT_saved_models\\best_model_epoch=33-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v45.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v46.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v47.ckpt', '.\\TFT_saved_models\\best_model_epoch=32-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v48.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v10.ckpt']

In [20]:
start_prediction_idx = 713
end_prediction_idx = 802
prediction_length = 1  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []


In [21]:
# Used for creating the validation set
training = TimeSeriesDataSet(
    data[(data.time_idx >= 1) & (data.time_idx <= 712)],
    time_idx="time_idx",
    target="Brent_IMF1",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["month","weekday","day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "Brent_IMF1",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

In [22]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

IMF1_fore=all_predictions

In [23]:
# IMF2

In [27]:
pretrained_model_paths=['.\\TFT_saved_models\\best_model_epoch=22-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=19-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=6-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=10-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v131.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v34.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=31-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v132.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=19-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=36-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v49.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v16.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=0-v35.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v133.ckpt', '.\\TFT_saved_models\\best_model_epoch=37-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=5-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=8-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v15.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v23.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v24.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v134.ckpt', '.\\TFT_saved_models\\best_model_epoch=5-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=2-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v25.ckpt', '.\\TFT_saved_models\\best_model_epoch=31-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v135.ckpt', '.\\TFT_saved_models\\best_model_epoch=35-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v26.ckpt', '.\\TFT_saved_models\\best_model_epoch=37-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=6-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v50.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v52.ckpt', '.\\TFT_saved_models\\best_model_epoch=45-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=6-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v53.ckpt', '.\\TFT_saved_models\\best_model_epoch=42-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=20-v1.ckpt', '.\\TFT_saved_models\\best_model_epoch=22-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=20-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v17.ckpt', '.\\TFT_saved_models\\best_model_epoch=20-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v16.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=7-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v17.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v18.ckpt', '.\\TFT_saved_models\\best_model_epoch=2-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=20-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=26-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=43-v18.ckpt', '.\\TFT_saved_models\\best_model_epoch=33-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=21-v4.ckpt', '.\\TFT_saved_models\\best_model_epoch=26-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v54.ckpt', '.\\TFT_saved_models\\best_model_epoch=2-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=35-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=27-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=44-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v138.ckpt', '.\\TFT_saved_models\\best_model_epoch=44-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=46-v27.ckpt', '.\\TFT_saved_models\\best_model_epoch=18-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=23-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v15.ckpt']

In [28]:
start_prediction_idx = 713
end_prediction_idx = 802
prediction_length = 1  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []


In [29]:
# Used for creating the validation set
training = TimeSeriesDataSet(
    data[(data.time_idx >= 1) & (data.time_idx <= 712)],
    time_idx="time_idx",
    target="Brent_IMF2",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["month","weekday","day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "Brent_IMF2",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

In [30]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

IMF2_fore=all_predictions

In [31]:
# Residual

In [32]:
pretrained_model_paths=['.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=27-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=25-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=25-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v17.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=3-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=30-v15.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v18.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v19.ckpt', '.\\TFT_saved_models\\best_model_epoch=13-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=40-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v20.ckpt', '.\\TFT_saved_models\\best_model_epoch=14-v2.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=19-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=15-v7.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=49-v141.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=28-v20.ckpt', '.\\TFT_saved_models\\best_model_epoch=21-v5.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=14-v3.ckpt', '.\\TFT_saved_models\\best_model_epoch=19-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=31-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=33-v11.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=16-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=37-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=32-v9.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=26-v10.ckpt', '.\\TFT_saved_models\\best_model_epoch=4-v6.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=37-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=34-v21.ckpt', '.\\TFT_saved_models\\best_model_epoch=38-v12.ckpt', '.\\TFT_saved_models\\best_model_epoch=27-v13.ckpt', '.\\TFT_saved_models\\best_model_epoch=2-v8.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=47-v14.ckpt', '.\\TFT_saved_models\\best_model_epoch=48-v64.ckpt']

In [33]:
start_prediction_idx = 713
end_prediction_idx = 802
prediction_length = 1  # The step length for each prediction
max_encoder_length=30

# Initialize an empty list to store all predictions
all_predictions = []


In [34]:
# Used for creating the validation set
training = TimeSeriesDataSet(
    data[(data.time_idx >= 1) & (data.time_idx <= 712)],
    time_idx="time_idx",
    target="Brent_Res",
    min_encoder_length=max_encoder_length // 2, 
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=prediction_length,
    time_varying_known_categoricals=["month","weekday","day"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "Brent_Res",
        
    ],
    group_ids=['destination'],
    target_normalizer=GroupNormalizer(
        groups=['destination'], transformation="softplus"),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,

   
)

In [35]:
# Loop for making predictions
for i, model_path in enumerate(pretrained_model_paths):
    start_idx = start_prediction_idx + i * prediction_length
    end_idx = start_idx + prediction_length

    if end_idx > end_prediction_idx:
        end_idx = end_prediction_idx + 1  
    
    # Create a new validation dataset
    validation_data = data[data['time_idx'] < end_idx].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    # Load the pretrained model and make predictions
    best_tft = TemporalFusionTransformer.load_from_checkpoint(model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    current_prediction = raw_predictions[0][:, :, 3]  # Retrieve current prediction results
    current_prediction_numpy = current_prediction.numpy().flatten()
    all_predictions.extend(current_prediction_numpy.tolist())

Res_fore=all_predictions

In [36]:
true_values = [87.86, 91.37, 90.7, 87.58, 88.4, 94.33, 90.99, 92.52, 91.99, 93.12, 93.72, 91.88, 88.0, 90.14, 88.45, 90.73, 90.73, 86.82, 86.92, 89.02, 87.55, 87.31, 83.43, 81.46, 81.74, 83.66, 84.09, 84.2, 82.4, 77.73, 81.22, 83.25, 82.42, 81.76, 80.85, 79.82, 79.49, 81.66, 82.98, 81.72, 78.72, 78.16, 77.27, 74.33, 74.21, 75.94, 75.75, 74.11, 74.14, 77.05, 76.84, 78.89, 79.82, 81.1, 80.73, 80.23, 80.6, 80.97, 79.04, 77.69, 76.24, 77.18, 75.79, 78.31, 75.47, 77.97, 78.46, 80.21, 79.89, 79.76, 80.15, 78.88, 81.04, 80.71, 81.7, 82.04, 82.15, 82.33, 83.34, 83.99, 84.14, 82.98, 82.2, 79.54, 79.3, 80.46, 81.18, 83.01, 83.58, 83.88]

In [37]:
import numpy as np

# Combine IMF1, IMF2, and residual components to get the final forecast
final = [sum(x) for x in zip(IMF1_fore, IMF2_fore, Res_fore)]
# Adjust the elements in the 'final' list to balance the previous increments of 10 each to IMF1 and IMF2, 
# effectively removing the added 20 units per element.
final = [x - 20 for x in final]

# Calculate Mean Absolute Error (MAE), Root Mean Square Error (RMSE), and Mean Absolute Percentage Error (MAPE)
mae = np.mean(np.abs(np.array(true_values) - np.array(final)))
rmse = np.sqrt(np.mean(np.square(np.array(true_values) - np.array(final))))
mape = np.mean(np.abs(np.array(true_values) - np.array(final)) / np.array(true_values))

# Print the evaluation metrics
print("MAE:", mae)
print("RMSE:", rmse)
print("MAPE:", mape)

MAE: 0.7199421106974282
RMSE: 1.0528983383515984
MAPE: 0.008829412043259627


## Part 2: Model Training

In [ ]:
# Before training the Temporal Fusion Transformer (TFT) model, 
# it is beneficial to employ the Tree-structured Parzen Estimator (TPE) method for hyperparameter optimization.
# This step aids in selecting the optimal hyperparameters for the TFT model. 
# However, the TPE method can be computationally intensive and involves randomness, 
# hence it is optional. We have already incorporated the optimal hyperparameters
# obtained from running this method into the corresponding TFT model.
# If you wish to perform this optimization and discover your own optimal hyperparameters,
# you may run the following code. Note that this process may significantly impact computational resources.

In [ ]:
# max_prediction_length = 1
# max_encoder_length = 30

# training = TimeSeriesDataSet(
#     data[(data.time_idx >= 1) & (data.time_idx <= 712)],
#     time_idx="time_idx",
#     target="Brent_IMF1",
#     min_encoder_length=max_encoder_length // 2, 
#     max_encoder_length=max_encoder_length,
#     min_prediction_length=1,
#     max_prediction_length=max_prediction_length,
#     time_varying_known_categoricals=["month","weekday","day"],
#     time_varying_known_reals=["time_idx"],
#     time_varying_unknown_categoricals=[],
#     time_varying_unknown_reals=[
#         "Brent_IMF1",
        
#     ],
#     group_ids=['destination'],
#     target_normalizer=GroupNormalizer(
#         groups=['destination'], transformation="softplus"),
#     add_relative_time_idx=True,
#     add_target_scales=True,
#     add_encoder_length=True,
#     allow_missing_timesteps=True,

   
# )
# validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)


# # create dataloaders for model
# batch_size = 256  # set this between 32 to 128


# # check if GPU is available
# if torch.cuda.is_available():
#     device = "cuda"
# else:
#     device = "cpu"

# # move dataloaders to device


# train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
# val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size*10 , num_workers=0)

In [ ]:
# import pickle
# from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters



# # create study
# study = optimize_hyperparameters(
#     train_dataloader,
#     val_dataloader,
#     model_path="optuna_test",
#     n_trials=50,
#     max_epochs=50,
#     gradient_clip_val_range=(0.01, 1.0),
#     hidden_size_range=(8, 128),
#     hidden_continuous_size_range=(8, 128),
#     attention_head_size_range=(1, 4),
#     learning_rate_range=(0.001, 0.1),
#     dropout_range=(0.1, 0.3),
#     trainer_kwargs=dict(limit_train_batches=30),
#     reduce_on_plateau_patience=4,
#     use_learning_rate_finder=False
# )


# # save study results - also we can resume tuning at a later point in time
# with open("test_study.pkl", "wb") as fout:
#     pickle.dump(study, fout)

# # show best hyperparameters
# print(study.best_trial.params)


In [5]:
# IMF 1

# Define the time index range for the validation dataset
start_prediction_idx = 713 # Starting index for prediction
end_prediction_idx = 802 # Ending index for prediction


# Set the prediction length to 1, indicating that the model predicts 1 steps ahead
prediction_length = 1

# Note: The test set comprises 90 time points. The prediction_length of 1 means that 
# the models trained in this session are designed to forecast 1 steps ahead. The 
# start and end indices (713 to 802) specify the time points covered by the 
# model's predictions. These indices also determine the number of models to be trained, 
# calculated as: (end_prediction_idx - start_prediction_idx + 1) / prediction_length.
# For start_prediction_idx = 713 and end_prediction_idx = 802 with prediction_length = 1,
# the total number of models trained in the loop is 90.

max_encoder_length = 30 

# Initialize an empty list to store all prediction values
all_predictions = []

# Initialize an empty list to store the best models for each prediction
all_best_models = []

In [6]:
# Caution: The models are resource-intensive, requiring a high-performance GPU. Training 
# too many models in a single loop may lead to exhaustion of computer resources, causing 
# the program to crash. In such cases, you may need to restart the program or adjust the 
# start and end indices to reduce the number of models trained in the loop.

In [ ]:
for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  
        time_idx="time_idx",
        target="Brent_IMF1",
        min_encoder_length=max_encoder_length // 2, 
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "Brent_IMF1",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )

    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    
    # configure network and trainer
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}", 
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.12606822464127176,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.0037662293387837106,
        hidden_size=8,
        attention_head_size=1,
        dropout=0.11405196829177033,
        hidden_continuous_size=8,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [ ]:
all_best_models

In [8]:
# IMF 2

In [9]:
start_prediction_idx = 713
end_prediction_idx = 802 

prediction_length = 1
max_encoder_length = 30 

all_predictions = []
all_best_models = []

In [ ]:
for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  
        time_idx="time_idx",
        target="Brent_IMF2",
        min_encoder_length=max_encoder_length // 2, 
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "Brent_IMF2",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )

    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1,  
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.015731346875597366,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.08854054714306236,
        hidden_size=41,
        attention_head_size=2,
        dropout=0.25541380275080905,
        hidden_continuous_size=14,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")



    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [ ]:
all_best_models

In [ ]:
# Residual

In [11]:
start_prediction_idx = 713
end_prediction_idx = 802 

prediction_length = 1
max_encoder_length = 30 

all_predictions = []
all_best_models = []

In [ ]:
for prediction_idx in range(start_prediction_idx, end_prediction_idx + 1, prediction_length):
    training = TimeSeriesDataSet(
        data[lambda x: x.time_idx <= prediction_idx - 1],  
        time_idx="time_idx",
        target="Brent_Res",
        min_encoder_length=max_encoder_length // 2, 
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=prediction_length,
        time_varying_known_categoricals=["month","weekday","day"],
        time_varying_known_reals=["time_idx"],
        time_varying_unknown_categoricals=[],
        time_varying_unknown_reals=[
            "Brent_Res",

        ],
        group_ids=['destination'],
        target_normalizer=GroupNormalizer(
            groups=['destination'], transformation="softplus"),

        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
    )

    validation_data = data[lambda x: x.time_idx <= prediction_idx - 1 + prediction_length].copy()
    validation = TimeSeriesDataSet.from_dataset(training, validation_data, predict=True, stop_randomization=True)
    train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)
    
    

    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        mode="min",
        save_last=True,
        save_top_k=1, 
        filename="best_model_{epoch}",  
        dirpath="saved_models"
    )

    early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
    lr_logger = LearningRateMonitor()  # log the learning rate
    logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

    trainer = pl.Trainer(
        max_epochs=50,
        gpus=1,
        enable_model_summary=True,
        gradient_clip_val=0.4804651992814277,
        limit_train_batches=30,  # coment in for training, running valiation every 30 batches
        # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
        callbacks=[lr_logger, early_stop_callback,checkpoint_callback],
        logger=logger,
    )


    tft = TemporalFusionTransformer.from_dataset(
        training,
        learning_rate=0.05225339638821306,
        hidden_size=37,
        attention_head_size=1,
        dropout=0.2747736782762395,
        hidden_continuous_size=20,
        output_size=7,  # 7 quantiles by default
        loss=QuantileLoss(),
        log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
        reduce_on_plateau_patience=4,
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


    
    
    trainer.fit(
        tft,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
    )


    best_model_path = trainer.checkpoint_callback.best_model_path
    current_model=best_model_path
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)
    all_best_models.append(current_model)
    current_prediction = raw_predictions[0][:, :, 3]  
    all_predictions.append(current_prediction)


In [ ]:
all_predictions 

In [ ]:
all_best_models